# Mixed ANOVA

Wir haben uns jetzt Schritt für Schritt die Fälle einer ANOVA mit einem einzelnen Faktor angesehen. In euren Arbeiten sind aber häufig mehrere Faktoren nötig. Außerdem werden sowohl between als auch within Faktoren benötigt. Daher sehen wir uns jetzt diesen Fall an. Hierfür laden wir auch erstmal den Pinguin Datensatz und fügen unsere zweiten und dritten Messzeitpunkte hinzu.

In [3]:
import pandas as pd

penguins = pd.read_csv("./penguins_classification.csv")

penguins.head()

,Culmen Length (mm),Culmen Depth (mm),Species
0,39.1,18.7,Adelie
1,39.5,17.4,Adelie
2,40.3,18.0,Adelie
3,36.7,19.3,Adelie
4,39.3,20.6,Adelie


In [4]:
## generate t2 and t3

import numpy as np
# np.random.rand generiert einen zufälligen Vektor mit den in den Klammern angegebenen Dimensionen
penguins_rm = pd.DataFrame({
    "id": np.resize(np.arange(0, len(penguins)), 3*len(penguins)), # Id erstellen und dreimal wiederholen, da wir drei Messzeitpunkte haben
    "time": ["t1"] * len(penguins) + ["t2"] * len(penguins) + ["t3"] * len(penguins), # 
    "Culmen Length (mm)": pd.concat([
        penguins["Culmen Length (mm)"],
        penguins["Culmen Length (mm)"] + np.random.rand(len(penguins)),
        penguins["Culmen Length (mm)"] + 0.5 + np.random.rand(len(penguins))
        ]),
    "Culmen Depth (mm)": pd.concat([
        penguins["Culmen Depth (mm)"],
        penguins["Culmen Depth (mm)"] + 2.63 * np.random.rand(len(penguins)),
        penguins["Culmen Depth (mm)"] + 7.94 * np.random.rand(len(penguins))
        ]),
    "Species": np.resize(penguins["Species"], 3*len(penguins))
})

# Addiere einen random Wert zu Gentoo zu t3, um einen Interaktionseffekt zu kreieren.
penguins_rm.loc[(penguins_rm["Species"] == "Gentoo") & (penguins_rm["time"] == "t3"), "Culmen Length (mm)"] += 2 * np.random.rand(1)

penguins_rm.groupby(["Species", "time"]).agg(["mean", "std"]).drop(columns=["id"])

Culmen Length (mm)           Culmen Depth (mm)          
                             mean       std              mean       std
Species   time                                                         
Adelie    t1            38.791391  2.663405         18.346358  1.216650
          t2            39.306395  2.682953         19.710511  1.529459
          t3            39.796739  2.652286         22.402870  2.376191
Chinstrap t1            48.833824  3.339256         18.420588  1.135395
          t2            49.329867  3.327058         19.668432  1.379190
          t3            49.875526  3.342314         22.286353  2.639263
Gentoo    t1            47.504878  3.081857         14.982114  0.981220
          t2            47.984348  3.100881         16.404182  1.168002
          t3            50.138105  3.084260         18.900750  2.400177

Wir sehen schon mal an der Tabelle, dass das ganze etwas aufwendiger wird, wenn wir mehrere Faktoren abbilden wollen. Trotzdem lässt sich mit `groupby` und `agg` einfach eine Tabelle erstellen. Die Liste an Tabellen Spalten für `groupby` wird nacheinander umgesetzt, genauso wie die Liste für `agg`. Die Durchführung der `mixed_anova` können wir uns zuerst einmal wieder in der Dokumentation von `pingouin` ansehen.

In [5]:
import pingouin as pg

?pg.mixed_anova

Signature:
pg.mixed_anova(
    data=None,
    dv=None,
    within=None,
    subject=None,
    between=None,
    correction='auto',
    effsize='np2',
)
Docstring:
Mixed-design (split-plot) ANOVA.

Parameters
----------
data : :py:class:`pandas.DataFrame`
    DataFrame. Note that this function can also directly be used as a
    Pandas method, in which case this argument is no longer needed.
dv : string
    Name of column containing the dependent variable.
within : string
    Name of column containing the within-subject factor
    (repeated measurements).
subject : string
    Name of column containing the between-subject identifier.
between : string
    Name of column containing the between factor.
correction : string or boolean
    If True, return Greenhouse-Geisser corrected p-value.
    If `'auto'` (default), compute Mauchly's test of sphericity to
    determine whether the p-values needs to be corrected.
effsize : str
    Effect size. Must be one of 'np2' (partial eta-squared), 'n2'


In [6]:
results_mx_anova = pg.mixed_anova(
    data=penguins_rm,
    dv="Culmen Length (mm)",
    within="time",
    subject="id",
    between="Species",
)
results_mx_anova.round(3)

,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,Species,23064.197,2,339,11532.098,439.713,0.0,NaN,0.722,NaN,NaN,NaN,NaN
1,time,457.287,2,678,228.643,4101.565,0.0,0.0,0.924,0.582,False,0.281,0.0
2,Interaction,139.861,4,678,34.965,627.234,0.0,NaN,0.787,NaN,NaN,NaN,NaN


Auffällig ist, dass wir hier keinen Fehler in einer Einzelnen Reihe ausgegeben bekommen, aber trotzdem Fehler-Freiheitsgrade (`DF2`). Außerdem sollte nach dem letzten Notebook klar sein, dass wir nur für den within-Factor Werte für die Tests bekommen, die Sphärizität messen und dafür korrigieren.

In [26]:
M_total = penguins_rm["Culmen Length (mm)"].mean()

SS_total = np.sum(( penguins_rm["Culmen Length (mm)"] - M_total) ** 2)
SS_total

32589.898299594974

In [27]:
def SumOfSquares(x):
    Mx = x.mean()
    return np.sum(( x - Mx) ** 2)

In [28]:
SS_Adelie = SumOfSquares(penguins_rm[penguins_rm["Species"] == "Adelie"]["Culmen Length (mm)"])
SS_Gentoo = SumOfSquares(penguins_rm[penguins_rm["Species"] == "Gentoo"]["Culmen Length (mm)"])
SS_Chinstrap = SumOfSquares(penguins_rm[penguins_rm["Species"] == "Chinstrap"]["Culmen Length (mm)"])

SS_within = SS_Adelie + SS_Gentoo + SS_Chinstrap
SS_within

9525.701792042028

In [29]:
SS_between = SS_total - SS_within

print(f"SS_between errechnet: {SS_between:.3f}\n"
      f"SS_between aus der Tabelle: {results_mx_anova['SS'][0]:.3f}")

SS_between errechnet: 23064.197
SS_between aus der Tabelle: 23064.197


In [30]:
M_group = penguins_rm.groupby("Species").mean()["Culmen Length (mm)"]
N_group = penguins_rm.groupby("Species").count()["Culmen Length (mm)"]

SS_between2 = np.sum( N_group * (M_group - M_total) ** 2)
SS_between2

23064.19650755298

In [32]:
# Lösung
subj_means = penguins_rm.groupby("id").mean()["Culmen Length (mm)"]
k = penguins_rm["time"].nunique()
SS_subj = np.sum( k * (subj_means - M_total) ** 2)
SS_subj

31954.954881174475

In [33]:
SS_error = SS_total - SS_subj
SS_error

634.9434184204983